### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2025-11-02 08:54:31--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2025-11-02 08:54:31--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2961d0c4e891ec30aa69033267.dl.dropboxusercontent.com/cd/0/inline/C0bQrT4FjC7yE4nEQx7wga2JzwMn6cMSn20fb20uOF4lxQ03r5lbvZYNFn-Z99Zs83MK7bU3aDuvnrVlxbuyvQyLXvCQOSh_UKLRWqockKbq3ZYkhLshyKMrvN6blrcqEAc/file?dl=1# [following]
--2025-11-02 08:54:31--  https://uc2961d0c4e891ec30aa69033267.dl.dropboxuse

,author,day,id,link,month,summary,tag,title,year
33981,[{'name': 'Mitsutaka Matsumoto'}],5,cmp-lg/9407004v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,(This is not the abstract): The language is Ja...,"[{'term': 'cmp-lg', 'scheme': 'http://arxiv.or...",Japanese word sense disambiguation based on ex...,1994
9431,"[{'name': 'Zackory Erickson'}, {'name': 'Henry...",27,1709.09735v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Robot-assisted dressing offers an opportunity ...,"[{'term': 'cs.RO', 'scheme': 'http://arxiv.org...",Deep Haptic Model Predictive Control for Robot...,2017
9711,"[{'name': 'Kuang Gong'}, {'name': 'Jiahui Guan...",9,1710.03344v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,PET image reconstruction is challenging due to...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Iterative PET Image Reconstruction Using Convo...,2017
4966,"[{'name': 'Hamidreza Alvari'}, {'name': 'Paulo...",30,1705.10786v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Human trafficking is one of the most atrocious...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Semi-Supervised Learning for Detecting Human T...,2017
18906,"[{'name': 'Rina Dechter'}, {'name': 'David Eph...",10,1301.2265v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,This paper explores algorithms for processing ...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Hybrid Processing of Beliefs and Constraints,2013


In [33]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [34]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

from nltk.tokenize import WordPunctTokenizer

tok = WordPunctTokenizer()
lines = [' '.join(tok.tokenize(s.lower())) for s in lines]


In [35]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [36]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    """
    counts = defaultdict(Counter)

    for line in lines:
        tokens = line.split()
        # добавляем EOS в конец каждой строки
        tokens = tokens + [EOS]

        for i, next_token in enumerate(tokens):
            # берем (n-1) предыдущих токенов
            start = max(0, i - (n - 1))
            prefix_tokens = tokens[start:i]
            # паддинг слева UNK
            if len(prefix_tokens) < (n - 1):
                prefix_tokens = [UNK] * ((n - 1) - len(prefix_tokens)) + prefix_tokens
            # если n == 1 → пустой префикс ()
            prefix = tuple(prefix_tokens[-(n - 1):]) if n > 1 else tuple()
            counts[prefix][next_token] += 1

    return counts



In [37]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [38]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
                # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        for prefix, token_counts in counts.items():
            total = sum(token_counts.values())
            if total > 0:
                self.probs[prefix] = {t: c / total for t, c in token_counts.items()}
            else:
                self.probs[prefix] = {}


    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [39]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [40]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [41]:

def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    """
    token_probs = lm.get_possible_next_tokens(prefix)

    # если модель вернула пусто — останавливаемся
    if not token_probs:
        return EOS

    tokens = list(token_probs.keys())
    probs = np.array([token_probs[t] for t in tokens], dtype=float)

    if temperature == 0:
        # жадный выбор
        return tokens[int(np.argmax(probs))]

    # temperature scaling
    scaled = np.power(probs, 1.0 / float(temperature))
    if scaled.sum() == 0:
        return tokens[int(np.argmax(probs))]
    scaled = scaled / scaled.sum()

    return np.random.choice(tokens, p=scaled)


In [42]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [43]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial agents ' characteristics , we design an agent . the framework of a well - known sylvester equation capturing both coarse and fine - tuning across all metrics while also offering substantial advantages of asymptotic optimality in the image based techniques . furthermore , we compare the recovery process that involves pseudo - boolean optimization show that the evaluation function of environmental factors as well as synthesize photorealistic textures , and mental state recognition from images , with the objective of this research , we proposed a formulation of the original loss function called range loss for the new additions


In [44]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - stream network architecture for the task of fine - tune it to a constant factor , and that the new method for learning bayesian networks . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [45]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_logprob = 0.0
    total_tokens = 0

    n = lm.n

    for line in lines:
        tokens = line.split()
        # последовательно прогнозируем каждое слово + EOS
        seq = tokens + [EOS]

        for i, w in enumerate(seq):
            # берем (n-1) предыдущих токенов для префикса
            start = max(0, i - (n - 1))
            prefix_tokens = tokens[start:i]
            if len(prefix_tokens) < (n - 1):
                prefix_tokens = [UNK] * ((n - 1) - len(prefix_tokens)) + prefix_tokens
            prefix = ' '.join(prefix_tokens) if prefix_tokens else ''

            p = lm.get_next_token_prob(prefix, w)
            # лог-вероятность с отсечением
            logp = np.log(p) if p > 0 else -np.inf
            if not np.isfinite(logp) or logp < min_logprob:
                logp = min_logprob

            total_logprob += logp
            total_tokens += 1

    # cross-entropy → perplexity
    avg_neg_logprob = - total_logprob / max(1, total_tokens)
    return float(np.exp(avg_neg_logprob))


In [46]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [30]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [47]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [48]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [49]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [50]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1832.66878
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [51]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [52]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        assert n >= 1
        self.n = n
        self.delta = float(delta)

        # считаем все порядки 1..n
        self.counts = {}
        for k in range(1, n + 1):
            self.counts[k] = count_ngrams(lines, n=k)

        # словарь (по униграммам)
        # counts[1] имеет префикс () → Counter({w: c})
        self.vocab = set(self.counts[1][tuple()].keys())

        # continuation probabilities для униграмм:
        # P_cont(w) = (# контекстов, где встречается bigram (*, w)) / (общее # различных биграмм)
        if n >= 2:
            bigram_counts = self.counts[2]  # префиксы длины 1
            # число различных биграмм
            total_bigram_types = sum(len(c) for c in bigram_counts.values())
            # для каждого w — количество различных префиксов, где w встречался
            cont_num = Counter()
            for prefix, cnts in bigram_counts.items():
                for w in cnts.keys():
                    cont_num[w] += 1
            self.unigram_cont_prob = {
                w: (cont_num[w] / total_bigram_types) if total_bigram_types > 0 else 0.0
                for w in self.vocab
            }
        else:
            # если n == 1, fallback: нормализуем частоты
            uni = self.counts[1][tuple()]
            total = sum(uni.values())
            self.unigram_cont_prob = {w: (uni[w] / total) if total > 0 else 0.0 for w in self.vocab}

    # вспомогательные функции
    def _pad_prefix(self, prefix_tokens, order):
        need = max(0, (order - 1) - len(prefix_tokens))
        if need > 0:
            prefix_tokens = [UNK] * need + prefix_tokens
        return tuple(prefix_tokens[-(order - 1):]) if order > 1 else tuple()

    def _P_kn(self, prefix_tokens, w, order):
        # базовый случай — униграммы
        if order <= 1:
            return self.unigram_cont_prob.get(w, 0.0)

        d = self.delta
        counts_k = self.counts[order]
        h = self._pad_prefix(prefix_tokens, order)

        token_counts = counts_k.get(h, Counter())
        c_hw = token_counts.get(w, 0)
        c_hstar = sum(token_counts.values())
        n1_hstar = len(token_counts)

        if c_hstar == 0:
            # если такой контекст не встречался — полностью бэк-оффим
            return self._P_kn(prefix_tokens[1:], w, order - 1)

        # нижний порядок с суффиксом контекста
        p_lower = self._P_kn(prefix_tokens[1:], w, order - 1)

        # абсолютное дисконтирование
        first_term = max(c_hw - d, 0.0) / c_hstar
        lambda_h = (d * n1_hstar) / c_hstar
        return first_term + lambda_h * p_lower

    def get_possible_next_tokens(self, prefix):
        # можно вернуть распределение по всему словарю,
        # чтобы сумма точно была 1 и тесты проходили
        prefix_tokens = prefix.split() if prefix else []
        probs = {w: self.get_next_token_prob(prefix, w) for w in self.vocab}
        # отфильтруем нули
        return {w: p for w, p in probs.items() if p > 0.0}

    def get_next_token_prob(self, prefix, next_token):
        if next_token not in self.vocab:
            return 0.0
        prefix_tokens = prefix.split() if prefix else []
        return float(self._P_kn(prefix_tokens, next_token, self.n))


In [53]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [54]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.75)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 385.61602
N = 3, Perplexity = 289.90797
